In [1]:
# -*- coding: utf-8 -*-

In [2]:
from __future__ import division, print_function, absolute_import

In [3]:
import tflearn
from tflearn.data_utils import shuffle
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.estimator import regression
from tflearn.data_preprocessing import ImagePreprocessing
from tflearn.data_augmentation import ImageAugmentation
import pickle

curses is not supported on this machine (please install/reinstall curses for an optimal experience)








In [4]:
#X, Y, X_test, Y_test = pickle.load(open('..\\Caps\\full_dataset.pkl', "rb")) ### 에러부분
data = open('..\\Caps\\full_dataset.pkl', "rb")
X, Y, X_test, Y_test = pickle.load(data, encoding='latin1')
###########수정 후, 정상 실행

In [5]:
X, Y = shuffle(X, Y)

In [6]:
img_prep = ImagePreprocessing()
img_prep.add_featurewise_zero_center()
img_prep.add_featurewise_stdnorm()

In [7]:
img_aug = ImageAugmentation()
img_aug.add_random_flip_leftright()
img_aug.add_random_rotation(max_angle=25.)
img_aug.add_random_blur(sigma_max=3.)

In [8]:
network = input_data(shape=[None, 32, 32, 3],
                     data_preprocessing=img_prep,
                     data_augmentation=img_aug)

In [9]:
network = conv_2d(network, 32, 3, activation='relu')

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.initializers.variance_scaling instead with distribution=uniform to get equivalent behavior.


In [10]:
network = max_pool_2d(network, 2)

In [11]:
network = conv_2d(network, 64, 3, activation='relu')

In [12]:
network = conv_2d(network, 64, 3, activation='relu')

In [13]:
network = max_pool_2d(network, 2)

In [14]:
##### 컨볼루젼 레이어로 학습, relu 사용하여 최적화
network = fully_connected(network, 512, activation='relu')

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [15]:
network = dropout(network, 0.5)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [16]:
network = fully_connected(network, 2, activation='softmax')

In [17]:
network = regression(network, optimizer='adam',
                     loss='categorical_crossentropy',
                     learning_rate=0.001)


Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [18]:
model = tflearn.DNN(network, tensorboard_verbose=0, checkpoint_path='..\\Caps\\bird-classifier\\bird-classifierbird-classifier.tfl.ckpt')


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



In [ ]:
model.fit(X, Y, n_epoch=100, shuffle=True, validation_set=(X_test, Y_test),
          show_metric=True, batch_size=96,
          snapshot_epoch=True,
          run_id='bird-classifier')

Training Step: 570  | total loss: 0.41727 | time: 42.572s
| Adam | epoch: 001 | loss: 0.41727 - acc: 0.8257 -- iter: 54720/56780


In [ ]:
##### 모델 save
model.save("bird-classifier.tfl")
print("Network trained and saved as bird-classifier.tfl!")